# Homework 1: Stance Classification

* 建立一个分类器，根据一段文字，预测pro vs. con
* 仅使用**主题'abortion'和'gay rights'**
* 在csv中，每行一个帖子。columns:帖子和信息，6个LIWC1特征，以及句子的3个词性标签的数量
* 用5-fold CV来评估模型，选择最佳features和每个主题的模型类型。 报告平均准确度和F1分，每个主题的前20个功能。
* 提交必须包含README文件
* 试验SVM和naive bayes
* Input: features. Output: label $y \in\{0,1\}$, “pro” (1) or “con” (0).

**每一个topic用两个模型**:
1. Ngrams: combination of unigrams, bigrams and trigrams as features.(实验找到最好的)
2. 对上面每个features set, 选模型SVM/NB. 也需要调参数防止overfitting (scikit-learn有包，Some examples are “kernel” and “C” for SVM.)

**Feature selection**
每组第二个模型可以考虑以下的feature或者额外的：
* Ngrams. Unigrams, bigrams, and trigrams.
* Cue words. The initial unigram, bigram, and trigram in a post.
* Repeated punctuation. Features to capture punctuation such as ??, ??????, !!!, or ?!.
* Part-of-speech tags. Features to count the number of nouns, verbs and adjectives in the
text (count_noun, count_verb, and count_adj).
* LIWC features. We give you a number of LIWC features in the data files. You may use
any combination of these as features.The measures we give and their descriptions are:
  * word_count: the number of words in the post.
  * words_pronom: the percent of words in the text that are pronominal.
  * words_per_sen: the average number of words per sentence in the text.
  * words_over_6: the percent of words in the text that are over 6 letters long.
  * pos_emo: the number of positive emotion words in the text.
  * neg_emo: the number of negative emotion words in the text.
* Any other features you come up with.

### 提交要求：
* 对于每个主题，请提供您最佳模型的书面说明，并提供合理的理由您使用或未使用的功能。您为什么认为他们有帮助？您
必须明确列出您在此处尝试过的功能。
* 对每个主题进行错误分析。为了获得最佳表现模型，请选择3个有趣的例子不起作用。指出为什么您认为它们的分类不正确。如果你可以添加其他功能，您认为哪些功能可以帮助提高性能？
对于每个示例，如果篇幅很长，则无需包括整个帖子文本，只需将您的讨论的相关部分和ID。
* 比较和对比您用于每个主题的功能。为什么你会想到一些
功能在不同主题下效果更好还是更差？还是您将相同的功能用于
所有主题，如果可以，为什么您认为效果很好？


### 评分
您将按照以下要素进行评分：
#### 2.1分类器（共60分）
您的交付品如下（50分）：
* 一个将火车文件和主题作为命令行参数的classify.py文件（即，可以作为“ python classify.py train-data.csv中止”运行），运行交叉验证选择模型，打印模型的平均精度和F1度量并打印顶部该主题的20个功能。 （60分）。
* 您的自述文件，如下所述（2.3）
* 准确性（10分）
您的系统将获得最高10分，具体取决于最佳模型是否产生了准确性高于或低于我们的阈值。
这些将在9月13日星期五之前发布在Piazza和课程网站上。

#### 2.2书面答案（30分）
* 为每个主题提供最佳性能模型的依据：说明完整且提供
有关每个模型（及其使用或不使用的功能）为何最好的信息。 （20分）
* 错误分析：所选职位应有所不同，说明应明确说明为什么功能无法预测正确的类别。 （10点）
您的可交付成果如下：
包含您的姓名，电子邮件地址，作业编号和
您对书面部分的回答。
* 2.3软件工程（包括文档）（10分）
您的自述文件必须包含以下内容：
您的姓名和电子邮件地址。
作业编号。
有关如何训练和测试分类器的信息。
分类器的特殊功能（或限制）的描述。
在代码文档中：
应该以有意义的方式记录代码。这可以表示表达功能/变量
名称以及评论。
信息性的方法/功能/变量名称。
高效实施。


### 提交说明
您应该在Courseworks上提交以下内容：
一个名为<YOUR-UNI> -hw1.zip的zip文件。这应该确切地列出了文件第2.1节中的可交付成果。您无需在zip中包含数据文件。
您应该在Gradescope上提交以下内容：
如第2.2节所述的hw1-write.pdf。

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import nltk
import pandas as pd

data = pd.read_csv('/content/gdrive/My Drive/NLP/stance-data.csv')
data.head()

,post_text,topic,author,label,id,word_count,words_pronom,words_per_sen,words_over_6,pos_emo,neg_emo,count_noun,count_verb,count_adj
0,There is only one question when it comes to ab...,abortion,jstantall,con,C0,185,18.50,1.62,3.78,16.22,12.97,41,33,15
1,People often try to personalize their pro-life...,abortion,NVYN,pro,C1,85,10.62,3.53,1.18,11.76,21.18,12,19,2
2,we 're saying that the pregnant person should ...,abortion,atypican,pro,C2,41,13.67,2.44,2.44,7.32,17.07,8,10,3
3,"No answer to my retort , just a down vote . Th...",abortion,atypican,pro,C3,12,6.00,8.33,0.00,0.00,16.67,3,2,1
4,I 'm sooo sick of people like you who feel lik...,abortion,SMCdeBater,con,C4,34,17.00,0.00,8.82,14.71,23.53,5,10,2
